<a href="https://colab.research.google.com/github/CalvHobbes/ai-agents/blob/main/Agentic_RAG_with_LLamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## REF - https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/2/router-query-engine

In [1]:
# Download the file dynamically from GitHub
!wget -O agentic_rag_llamaindex_requirements.txt https://raw.githubusercontent.com/CalvHobbes/ai-agents/main/agentic_rag_llamaindex_requirements.txt




--2025-01-22 11:14:27--  https://raw.githubusercontent.com/CalvHobbes/ai-agents/main/agentic_rag_llamaindex_requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252 [text/plain]
Saving to: ‘agentic_rag_llamaindex_requirements.txt’

agentic_rag_llamain 100%[===================>]     252  --.-KB/s    in 0s      

2025-01-22 11:14:28 (3.74 MB/s) - ‘agentic_rag_llamaindex_requirements.txt’ saved [252/252]



In [2]:
# Install the requirements
!pip install -r agentic_rag_llamaindex_requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00


In [ ]:
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

In [6]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)


In [8]:
import nest_asyncio

nest_asyncio.apply()

In [16]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAPI_KEY')

In [20]:

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

# Initialize OpenAI LLM and embedding model
# llm = OpenAI(model="gpt-3.5-turbo")
# embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

# Use these models in your index or service context
# service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

# Example usage: Create an index
# index = GPTSimpleVectorIndex([], service_context=service_context)


TypeError: Can't instantiate abstract class OpenAI with abstract method _prepare_chat_with_tools

In [22]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
# Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
# Settings.num_output = 512
# Settings.context_window = 3900

TypeError: Can't instantiate abstract class OpenAI with abstract method _prepare_chat_with_tools

In [24]:
from llama_index import SummaryIndex, VectorStoreIndex
from llama_index import ServiceContext, set_global_service_context
from llama_index.llms import OpenAI

# Initialize LLM
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
service_context = ServiceContext.from_defaults(llm=llm)
set_global_service_context(service_context)

# Load documents
documents = SimpleDirectoryReader('data').load_data()

# Example 1: Using SummaryIndex
def summary_index_example():
    # Create and build the index
    summary_index = SummaryIndex.from_documents(documents)

    # Query the index
    query_engine = summary_index.as_query_engine()
    response = query_engine.query(
        "What are the main topics covered in these documents?"
    )
    print("Summary Index Response:", response)

    # Get a hierarchical summary
    hierarchical_summary = summary_index.get_document_summary()
    print("Hierarchical Summary:", hierarchical_summary)

# Example 2: Using VectorStoreIndex
def vector_store_example():
    # Create and build the index
    vector_index = VectorStoreIndex.from_documents(documents)

    # Basic querying
    query_engine = vector_index.as_query_engine()
    response = query_engine.query(
        "Find relevant information about machine learning"
    )
    print("Vector Store Response:", response)

    # Advanced querying with similarity search
    retriever = vector_index.as_retriever(
        similarity_top_k=2  # Return top 2 most similar chunks
    )
    nodes = retriever.retrieve("machine learning")

    # Print similar chunks
    for node in nodes:
        print(f"Score: {node.score}")
        print(f"Content: {node.node.get_content()}\n")

# Example 3: Combining both indexes
def combined_index_example():
    # Create both indexes
    summary_index = SummaryIndex.from_documents(documents)
    vector_index = VectorStoreIndex.from_documents(documents)

    # Get high-level summary from SummaryIndex
    summary_query_engine = summary_index.as_query_engine()
    summary = summary_query_engine.query("Provide a brief overview")

    # Get specific details from VectorStoreIndex
    vector_query_engine = vector_index.as_query_engine()
    details = vector_query_engine.query("What are the technical details about...")

    print("Summary:", summary)
    print("Details:", details)

if __name__ == "__main__":
    print("Running SummaryIndex example...")
    summary_index_example()

    print("\nRunning VectorStore example...")
    vector_store_example()

    print("\nRunning Combined example...")
    combined_index_example()

ImportError: cannot import name 'SummaryIndex' from 'llama_index' (unknown location)